In [0]:
!git clone https://github.com/eth-sri/ERAN.git

In [0]:
%cd ERAN

### Install M4

In [0]:
!wget ftp://ftp.gnu.org/gnu/m4/m4-1.4.1.tar.gz
!tar -xvzf m4-1.4.1.tar.gz

In [0]:
%cd m4-1.4.1

In [0]:
!./configure

In [0]:
!make
!make install

In [0]:
!cp src/m4 /usr/bin

In [0]:
%cd ..

In [0]:
!rm m4-1.4.1.tar.gz

### Install gmp

In [0]:
!wget https://gmplib.org/download/gmp/gmp-6.1.2.tar.xz
!tar -xvf gmp-6.1.2.tar.xz

In [0]:
%cd gmp-6.1.2

In [0]:
!./configure --enable-cxx

In [0]:
!make
!make install

In [0]:
!make check

In [0]:
%cd ..

In [0]:
!rm gmp-6.1.2.tar.xz

### Install mpfr

In [0]:
!wget https://www.mpfr.org/mpfr-current/mpfr-4.0.2.tar.xz
!tar -xvf mpfr-4.0.2.tar.xz

In [0]:
%cd mpfr-4.0.2

In [0]:
!./configure

In [0]:
!make
!make install

In [0]:
%cd ..

In [0]:
!rm mpfr-4.0.2.tar.xz

### Install ELINA

In [0]:
!git clone https://github.com/eth-sri/ELINA.git

In [0]:
%cd ELINA

In [0]:
!./configure

In [0]:
!make
!make install

In [0]:
%cd ..

### Install Gurobi

In [0]:
!wget https://packages.gurobi.com/9.0/gurobi9.0.0_linux64.tar.gz
!tar -xvf gurobi9.0.0_linux64.tar.gz

In [0]:
%cd gurobi900/linux64/src/build

In [0]:
!sed -ie 's/^C++FLAGS =.*$/& -fPIC/' Makefile
!make

In [0]:
!cp libgurobi_c++.a ../../lib/

In [0]:
%cd ../../

In [0]:
!cp lib/libgurobi90.so /usr/lib
!python3 setup.py install

In [0]:
%cd ../../

### Update environment variables:

In [0]:
%env GUROBI_HOME=/content/ERAN/gurobi900/linux64
gurobi_home = %env GUROBI_HOME
%env PATH
path = %env PATH
path += ':' + gurobi_home
print(path)
%env PATH=$path
ld_library_path = %env LD_LIBRARY_PATH
ld_library_path += ':' + gurobi_home
print(ld_library_path)
%env LD_LIBRARY_PATH=$ld_library_path

### Install DeepG

In [0]:
!rm -r deepg
!git clone https://github.com/eth-sri/deepg.git

In [0]:
%cd deepg/code

In [0]:
!sed s/81/90/ <Makefile >Makefile2
!sed s/build/./ <Makefile2 >Makefile3
!rm Makefile Makefile2
!mv Makefile3 Makefile

In [0]:
!mkdir build
!make shared_object

In [0]:
!cp ./build/libgeometric.so /usr/lib
!cd ../..

In [0]:
!ldconfig


### Install other requirements


In [0]:
%cd ../../..

In [0]:
!sed "s/tensorflow/tensorflow<2/" <requirements.txt >requirements2.txt
!rm requirements.txt
!mv requirements2.txt requirements.txt
!pip3 install -r requirements.txt

### Test

In [0]:
!wget -O mnist.tf https://files.sri.inf.ethz.ch/eran/nets/tensorflow/mnist/mnist_relu_3_50.tf

In [0]:
%cd ERAN/tf_verify

In [0]:
!python3 . --netname ../../mnist.tf --epsilon 0.1 --domain deepzono --dataset mnist